**Writing Smart Contracts 2024: Final Project by Kristina Odermatt (odermk@usi.ch)**

**(6) Voting**:  A club has a membership list with the public keys of all members. Write a smart contract that allows members to vote for one of three candidates (Rossi, Smith or Meier). Extension: (1) Ensure that voting is only possible in a certain time frame.



# **Notebook 2: Using the smart contract for Club**

## Setup


In [21]:
# loading shared code and credentials
from algo_util import *

cred = load_credentials("./credentials_project")
members = cred['Members']
club = cred['Club']


In [22]:
# importing necessary modules and functions from the Algorand SDK
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk import transaction
from algosdk.transaction import PaymentTxn
from algosdk.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
import algosdk.error
import json
import base64
import datetime

In [23]:
# importing all the contents from the pyteal module
from pyteal import *

In [24]:
# Initialize the algod client (Testnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['api_token'])
algod_client.status()['last-round']

38927564

In [25]:
# define the app_id from '01_Create_SC'

app_id = 640731779                 # <--- ‼️‼️ INSERT/UPDATE APP_ID

## Step 1: Club opts-in

In [26]:
user = club

# Step 1: prepare transaction
sp = algod_client.suggested_params()
txn = transaction.ApplicationOptInTxn(user['public'], sp, app_id)

# Step 2: sign transaction
stxn = txn.sign(user['private'])

# Step 3: send
txid = algod_client.send_transactions([stxn])

# Step 4: await confirmation
txinfo = wait_for_confirmation(algod_client, txid)


Current round is  38927565.
Waiting for round 38927565 to finish.
Waiting for round 38927566 to finish.
Transaction DIIEI7HRTZ6U2URG7ICE5ECZTUUZ5S46CX3OZKCXY7IROPN6I3AA confirmed in round 38927567.


## Step 2: Inspecting the global state and the local states of the users
During the voting period the club account is able to inspect global and local state to 
* check how many memebers have voted
* check for each candidate's vote count
* check rounds

In [27]:
#Inspecting global state
read_global_state(algod_client,app_id) 

{'Rossi': 7,
 'StartRound': 38927105,
 'TotalVotes': 14,
 'EndRound': 38927305,
 'Smith': 3,
 'Meier': 4,
 'Members': 15,
 'Note': 'Welcome to the Voting System'}

In [28]:
# insepct local state for Members
member_name = 'Member_1'                    # <-- ‼️ UPDATE/CHANGE Member's name for each 

read_local_state(algod_client,members[member_name]['public'],app_id)

{'has_voted': 1}

In [29]:
# insepct local state Club
read_local_state(algod_client,club['public'],app_id)

{'has_voted': 0}

## Step 3: Club account closes out

In [30]:
user = club

# Step 1: Prepare the CloseOut transaction
sp = algod_client.suggested_params()
txn = transaction.ApplicationCloseOutTxn(user['public'], sp, app_id)

# Step 2: Sign the transaction with the club's private key
stxn = txn.sign(user['private'])

# Step 3: Send the transaction
txid = algod_client.send_transactions([stxn])

# Step 4: Await confirmation
txinfo = wait_for_confirmation(algod_client, txid)

print(f"Club closed out successfully. Transaction ID: {txid}")


Current round is  38927568.
Waiting for round 38927568 to finish.
Waiting for round 38927569 to finish.
Transaction ZZNJ6XYTSIZ5GTTYQYDCB4FQPIDAKSOWDPTUYS37TV5CTYZ6ITCA confirmed in round 38927570.
Club closed out successfully. Transaction ID: ZZNJ6XYTSIZ5GTTYQYDCB4FQPIDAKSOWDPTUYS37TV5CTYZ6ITCA



### Inspect the global state and the local states of the users

* Check if memebers account and club account have closed out

In [31]:
#Inspect global state
read_global_state(algod_client,app_id)

{'EndRound': 38927305,
 'StartRound': 38927105,
 'TotalVotes': 14,
 'Note': 'Welcome to the Voting System',
 'Meier': 4,
 'Smith': 3,
 'Members': 15,
 'Rossi': 7}

In [32]:
member_name = 'Member_15'        # <-- ‼️ UPDATE/CHANGE Member's name

# Inspecting local state for Members
read_local_state(algod_client,members[member_name]['public'],app_id)

{'has_voted': 0}

In [33]:
#Inspect local state for Club
read_local_state(algod_client,club['public'],app_id)

{}

## Step 4: Club account deletes the App

* The creator delets the App after the voting is concluded

*  ‼️ Before running the following step, please test the Smart Contract in Notebook '03_Project_WSC' for test 3.4 and 3.5

In [34]:
creator = club

# Step 1: Prepare transaction
sp = algod_client.suggested_params()
txn = transaction.ApplicationDeleteTxn(creator['public'], sp, app_id)

# Step 2: sign
stxn = txn.sign(creator['private'])

# Step 3: send
txid = algod_client.send_transactions([stxn])

# Step4 : wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

# display results
print("Deleted app-id:", txinfo["txn"]["txn"]["apid"])

Current round is  38927635.
Waiting for round 38927635 to finish.
Waiting for round 38927636 to finish.
Transaction 34525LUNMMNNXY4EDF2Y45E534JOXJEJYRGQRXVNPURD2W3B4CMQ confirmed in round 38927637.
Deleted app-id: 640731779
